In [ ]:
import os
import cv2
import kagglehub
from ultralytics import YOLO
import torch
import time
import pandas as pd
from tqdm import tqdm
from torchvision.models.detection import fasterrcnn_resnet50_fpn, ssd300_vgg16
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt
import json
from google.colab import files

# Create a results folder in Colab
RESULTS_FOLDER = '/content/object_detection_results'
os.makedirs(RESULTS_FOLDER, exist_ok=True)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 1: Download the 'indoor-object-detection' dataset from Kaggle
def download_kaggle_dataset():
    print("Downloading the 'indoor-object-detection' dataset from Kaggle...")
    path = kagglehub.dataset_download("thepbordin/indoor-object-detection")
    print("Dataset downloaded successfully.")
    print("Path to dataset files:", path)
    return path

# Step 2: Image Preprocessing (resize and normalize)
def preprocess_image(image, size=(640, 640)):
    img_resized = cv2.resize(image, size)
    img_normalized = img_resized / 255.0
    img_uint8 = (img_normalized * 255).astype('uint8')
    return img_uint8

# Function to save image in the results folder
def save_image(img, filename):
    full_path = os.path.join(RESULTS_FOLDER, filename)
    cv2.imwrite(full_path, img)
    return full_path

# Step 3: Process a single image with YOLOv8
def process_image_yolov8(image_path, conf_threshold=0.3):
    print(f"Processing image with YOLOv8: {image_path}")
    model = YOLO('yolov8x.pt').to(device)

    img = cv2.imread(image_path)
    img_preprocessed = preprocess_image(img)

    start_time = time.time()
    results = model(img_preprocessed, conf=conf_threshold, iou=0.5)
    inference_time = time.time() - start_time

    img_draw = img.copy()
    detection_counts = {}

    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = float(box.conf)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = result.names[int(box.cls)]
            detection_counts[label] = detection_counts.get(label, 0) + 1
            cv2.rectangle(img_draw, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label_conf = f"{label} {conf:.2f}"
            cv2.putText(img_draw, label_conf, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    processed_filename = 'processed_yolov8_' + os.path.basename(image_path)
    saved_path = save_image(img_draw, processed_filename)
    return saved_path, detection_counts, inference_time

# Step 4: Process a single image with Faster R-CNN
def process_image_faster_rcnn(image_path):
    print(f"Processing image with Faster R-CNN: {image_path}")
    model = fasterrcnn_resnet50_fpn(pretrained=True).to(device)
    model.eval()

    img = cv2.imread(image_path)
    img_preprocessed = preprocess_image(img)
    img_rgb = cv2.cvtColor(img_preprocessed, cv2.COLOR_BGR2RGB)
    img_tensor = F.to_tensor(img_rgb).unsqueeze(0).to(device)

    start_time = time.time()
    with torch.no_grad():
        predictions = model(img_tensor)
    inference_time = time.time() - start_time

    img_draw = img.copy()
    detection_counts = {}

    for box, label, score in zip(predictions[0]['boxes'], predictions[0]['labels'], predictions[0]['scores']):
        if score >= 0.3:
            x1, y1, x2, y2 = map(int, box.cpu().tolist())
            label_name = f"Label {label.item()}"
            detection_counts[label_name] = detection_counts.get(label_name, 0) + 1
            cv2.rectangle(img_draw, (x1, y1), (x2, y2), (0, 0, 255), 2)
            label_conf = f"{label_name} {score:.2f}"
            cv2.putText(img_draw, label_conf, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    processed_filename = 'processed_fasterrcnn_' + os.path.basename(image_path)
    saved_path = save_image(img_draw, processed_filename)
    return saved_path, detection_counts, inference_time

# Step 5: Process a single image with SSD (Single Shot MultiBox Detector)
def process_image_ssd(image_path, conf_threshold=0.3):
    print(f"Processing image with SSD: {image_path}")
    model = ssd300_vgg16(pretrained=True).to(device)
    model.eval()

    img = cv2.imread(image_path)
    img_preprocessed = preprocess_image(img)
    img_rgb = cv2.cvtColor(img_preprocessed, cv2.COLOR_BGR2RGB)
    img_tensor = F.to_tensor(img_rgb).unsqueeze(0).to(device)

    start_time = time.time()
    with torch.no_grad():
        predictions = model(img_tensor)
    inference_time = time.time() - start_time

    img_draw = img.copy()
    detection_counts = {}

    for box, score in zip(predictions[0]['boxes'], predictions[0]['scores']):
        if score >= conf_threshold:
            x1, y1, x2, y2 = map(int, box.cpu().tolist())
            detection_counts["SSD Object"] = detection_counts.get("SSD Object", 0) + 1
            cv2.rectangle(img_draw, (x1, y1), (x2, y2), (255, 0, 0), 2)
            label_conf = f"Object {score:.2f}"
            cv2.putText(img_draw, label_conf, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    processed_filename = 'processed_ssd_' + os.path.basename(image_path)
    saved_path = save_image(img_draw, processed_filename)
    return saved_path, detection_counts, inference_time

# Function to display and save comparison images
def display_images(original, yolo, rcnn, ssd, filename):
    plt.figure(figsize=(20, 5))

    plt.subplot(141)
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(142)
    plt.imshow(cv2.cvtColor(yolo, cv2.COLOR_BGR2RGB))
    plt.title('YOLOv8 Detection')
    plt.axis('off')

    plt.subplot(143)
    plt.imshow(cv2.cvtColor(rcnn, cv2.COLOR_BGR2RGB))
    plt.title('Faster R-CNN Detection')
    plt.axis('off')

    plt.subplot(144)
    plt.imshow(cv2.cvtColor(ssd, cv2.COLOR_BGR2RGB))
    plt.title('SSD Detection')
    plt.axis('off')

    plt.tight_layout()
    plot_path = os.path.join(RESULTS_FOLDER, f'comparison_plot_{filename}.png')
    plt.savefig(plot_path)
    plt.close()

# Function to save checkpoint
def save_checkpoint(dataset_dir, processed_files, results_list):
    checkpoint = {
        'dataset_dir': dataset_dir,
        'processed_files': processed_files,
        'results_list': results_list
    }
    checkpoint_path = os.path.join(RESULTS_FOLDER, 'checkpoint.json')
    with open(checkpoint_path, 'w') as f:
        json.dump(checkpoint, f)

# Function to load checkpoint
def load_checkpoint():
    checkpoint_path = os.path.join(RESULTS_FOLDER, 'checkpoint.json')
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'r') as f:
            return json.load(f)
    return None

# Step 6: Run object detection on images from the dataset with checkpointing and batching
def run_object_detection(dataset_dir, max_images=600, batch_size=10):
    checkpoint = load_checkpoint()
    if checkpoint and checkpoint['dataset_dir'] == dataset_dir:
        processed_files = set(checkpoint['processed_files'])
        results_list = checkpoint['results_list']
        print(f"Resuming from checkpoint. {len(processed_files)} images already processed.")
    else:
        processed_files = set()
        results_list = []

    image_files = [f for f in os.listdir(dataset_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    image_files = [f for f in image_files if f not in processed_files][:max_images]

    for i in tqdm(range(0, len(image_files), batch_size), desc="Processing Batches", unit="batch"):
        batch = image_files[i:i+batch_size]
        for filename in batch:
            image_path = os.path.join(dataset_dir, filename)
            original_image = cv2.imread(image_path)

            # YOLOv8
            processed_filename_yolo, detection_counts_yolo, yolo_time = process_image_yolov8(image_path)
            yolo_image = cv2.imread(processed_filename_yolo)

            # Faster R-CNN
            processed_filename_rcnn, detection_counts_rcnn, rcnn_time = process_image_faster_rcnn(image_path)
            rcnn_image = cv2.imread(processed_filename_rcnn)

            # SSD
            processed_filename_ssd, detection_counts_ssd, ssd_time = process_image_ssd(image_path)
            ssd_image = cv2.imread(processed_filename_ssd)

            # Display and save the comparison plot
            display_images(original_image, yolo_image, rcnn_image, ssd_image, filename)

            results_list.append({
                "Image": filename,
                "YOLOv8 Detections": detection_counts_yolo,
                "YOLOv8 Time (s)": yolo_time,
                "Faster R-CNN Detections": detection_counts_rcnn,
                "Faster R-CNN Time (s)": rcnn_time,
                "SSD Detections": detection_counts_ssd,
                "SSD Time (s)": ssd_time
            })

            processed_files.add(filename)

        # Save checkpoint after each batch
        save_checkpoint(dataset_dir, list(processed_files), results_list)

        # Save partial results to CSV in the results folder
        partial_csv_path = os.path.join(RESULTS_FOLDER, 'comparative_analysis_partial.csv')
        df_results = pd.DataFrame(results_list)
        df_results.to_csv(partial_csv_path, index=False)
        print(f"Partial results saved. Processed {len(processed_files)} images so far.")

    # Save final results to CSV in the results folder
    final_csv_path = os.path.join(RESULTS_FOLDER, 'comparative_analysis_final.csv')
    df_results = pd.DataFrame(results_list)
    df_results.to_csv(final_csv_path, index=False)
    print(f"Final comparative analysis saved to '{final_csv_path}'.")

# Main execution
if __name__ == "__main__":
    # Download and extract the dataset
    dataset_path = download_kaggle_dataset()

    # Define the paths for train, validation, and test images
    train_images_path = os.path.join(dataset_path, "train/images")
  

    # Run the object detection on each dataset
    for dataset_name, dataset_path in [
        ("train", train_images_path),
     
    ]:
        print(f"Starting object detection on {dataset_name} dataset...")
        run_object_detection(dataset_path, max_images=600, batch_size=10)

    # Zip and download the results folder
    !zip -r /content/object_detection_results.zip {RESULTS_FOLDER}
    files.download('/content/object_detection_results.zip')